In [ ]:
!pip uninstall ipywidgets
!pip install timm==0.6.7 pytorch-lightning==1.7.1

# Download Dataset

In [ ]:
# Setup Account file
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
%%bash 

mkdir /root/.kaggle
cp kaggle.json /root/.kaggle/

In [ ]:
!chmod 600 /root/.kaggle/kaggle.json

In [ ]:
# create new directory
![[ -d "../input" ]] && rm -r ../input
!mkdir -p ../input/mayo-clinic-strip-ai

In [ ]:
%%bash

kaggle competitions download -c mayo-clinic-strip-ai -p ../input/mayo-clinic-strip-ai -f train.csv
kaggle competitions download -c mayo-clinic-strip-ai -p ../input/mayo-clinic-strip-ai -f test.csv
kaggle competitions download -c mayo-clinic-strip-ai -p ../input/mayo-clinic-strip-ai -f sample_submission.csv

In [ ]:
%%bash

# DOWNLOAD TEST DIRECTORY

mkdir ../input/mayo-clinic-strip-ai/test

kaggle competitions files -c mayo-clinic-strip-ai | grep test/ | awk '{print $1}' \
 | while read x ; do kaggle competitions download -f $x mayo-clinic-strip-ai -p ../input/mayo-clinic-strip-ai/test ; done

for filename in ../input/mayo-clinic-strip-ai/test/*.zip; do
  unzip ${filename} -d ../input/mayo-clinic-strip-ai/test
  rm ${filename}
done

In [ ]:
%%bash

# Download dataset from kaggle
[[ ! -e /content/strip-ai-tiles.zip ]] && kaggle datasets download djagatiya/strip-ai-tiles
[[ ! -d "../input/strip-ai-tiles" ]] && unzip /content/strip-ai-tiles.zip -d ../input/strip-ai-tiles

In [ ]:
test_num = 1

# Training

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

from PIL import Image

from sklearn.utils import resample
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report

import pytorch_lightning as pl


import torch
import random

plt.style.use("dark_background")

In [ ]:
train_df = pd.read_csv("../input/mayo-clinic-strip-ai/train.csv")
test_df = pd.read_csv("../input/mayo-clinic-strip-ai/test.csv")
# other_df = pd.read_csv("../input/mayo-clinic-strip-ai/other.csv")

In [ ]:
train_df.head()

In [ ]:
test_df.head()

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=4, figsize=(20,4))

train_df.label.value_counts().plot(kind='bar', ax=axes[1], title='Train Target')
train_df.center_id.value_counts().plot(kind='bar', ax=axes[2], title='Centers')

number_img, num_patients = np.unique(train_df.patient_id.value_counts().values, return_counts=True)

sns.barplot(x=number_img, y=num_patients, ax=axes[3]).set_title('Images')

pd.DataFrame.from_dict([
    {
        "Type" : "Train",
        "count": len(train_df)
    },
    {
        "Type" : "Test",
        "count": len(test_df)
    },
    # {
    #     "Type" : "Other",
    #     "count": len(other_df)
    # }
]).set_index('Type').plot(kind='bar', ax=axes[0], title='Df Size')

plt.show()

In [ ]:
train_df['image_path'] = train_df.image_id.apply(lambda x : f"../input/mayo-clinic-strip-ai/train/{x}.tif")

In [ ]:
train_df.head()

In [ ]:
train_tiles_df = pd.read_csv("../input/strip-ai-tiles/train_df_tiles.csv")
train_tiles_df = train_tiles_df[['image_id','tile_ids']]

In [ ]:
train_tiles_df.head()

In [ ]:
train_df = pd.merge(train_df, train_tiles_df, on=['image_id'])
display(train_df.head())

In [ ]:
def display_tiles(row):
#     print(row)
    tile_ids = eval(row.tile_ids)
     
    fig, ax_ls = plt.subplots(1, len(tile_ids), figsize=(20,4))
    
    for i, tile_id in enumerate(tile_ids):
        path = f"../input/strip-ai-tiles/tiles/{row.image_id}.{tile_id}.png"
        img = Image.open(path)
        ax_ls[i].imshow(img)
        ax_ls[i].axis('off')
    
    plt.show()

In [ ]:
for i in range(5):
  display_tiles(train_df.iloc[i])

In [ ]:
train_df_CE = train_df[train_df.label == 'CE']
train_df_LAA = train_df[train_df.label == 'LAA']

train_df_CE.shape, train_df_LAA.shape

In [ ]:
train_df_LAA_upsampled = resample(train_df_LAA,
                                 replace=True,    # sample with replacement
                                 n_samples= 547, # to match majority class
                                 random_state=42)  # reproducible results

In [ ]:
train_df_LAA_upsampled.shape

In [ ]:
train_df = pd.concat([train_df_CE, train_df_LAA_upsampled])
train_df.shape

In [ ]:
train_df.label.value_counts()

In [ ]:
train_split_df, val_split_df = train_test_split(train_df, random_state=2022)
train_split_df.shape, val_split_df.shape

In [ ]:
train_split_df.label.value_counts()

In [ ]:
val_split_df.label.value_counts()

In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms

from torch.utils.data import Dataset, DataLoader

In [ ]:
train_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [ ]:
val_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [ ]:
class TilesDataset(Dataset):
    """Face Landmarks dataset."""

    def __init__(self, df, transform):
        self.df = df
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        
        row = self.df.iloc[idx]
        
        images = []
        
        tile_ids = eval(row.tile_ids)
        for tile_id in tile_ids:
            path = f"../input/strip-ai-tiles/tiles/{row.image_id}.{tile_id}.png"
            img = Image.open(path)
            images.append(img)
            
        images = [self.transform(i) for i in images]
        label = [0] if row.label == 'CE' else [1]
        
        images = torch.stack(images)
        
        label = torch.Tensor(label)
        
        return images, label

In [ ]:
train_split_ds = TilesDataset(train_split_df, train_transform)
print(len(train_split_ds))

demo_arr = train_split_ds[0][0]
demo_arr.shape, demo_arr.min(), demo_arr.max()

In [ ]:
import torch.nn as nn
import timm
import torch.optim as optim
import torch.nn.functional as F

In [ ]:
def get_positional_embedding(s_len, d):
    result = torch.ones(s_len, d)
    for i in range(s_len):
        for j in range(d):
            if j % 2 == 0:
                result[i][j] = np.sin(i / (10000 ** (j / d)))
            else:
                result[i][j] = np.cos(i / (10000 ** ((j-1) / d)))
    return result 

In [ ]:
# Remove cached timm models
!rm -r /root/.cache/torch/hub/checkpoints

In [ ]:
model = timm.create_model("efficientnet_b0", pretrained=True, num_classes=0)
for i, _children in enumerate(model.children()):
    print(i, type(_children))

In [ ]:
import math

def initialize_weights(m):
    if isinstance(m, nn.Linear):
        nn.init.normal_(m.weight, mean=0.0, std=0.01)
        # nn.init.kaiming_uniform_(m.weight.data)
        if m.bias is not None:
            nn.init.zeros_(m.bias)
    elif isinstance(m, nn.Parameter):
        nn.init.normal_(m, mean=0.0, std=0.01)

In [ ]:
class AttentionCust(nn.Module):
    def __init__(self, dim, num_heads=8, qkv_bias=False, attn_drop=0., proj_drop=0.):
        super().__init__()
        assert dim % num_heads == 0, 'dim should be divisible by num_heads'
        self.num_heads = num_heads
        head_dim = dim // num_heads
        self.scale = head_dim ** -0.5

        self.qkv = nn.Linear(dim, dim * 3, bias=qkv_bias)
        self.attn_drop = nn.Dropout(attn_drop)
        self.proj = nn.Linear(dim, dim)
        self.proj_drop = nn.Dropout(proj_drop)

        initialize_weights(self.qkv)
        initialize_weights(self.proj)


    def forward(self, x):
        B, N, C = x.shape
        qkv = self.qkv(x).reshape(B, N, 3, self.num_heads, C // self.num_heads).permute(2, 0, 3, 1, 4)
        q, k, v = qkv.unbind(0)   # make torchscript happy (cannot use tensor as tuple)

        attn = (q @ k.transpose(-2, -1)) * self.scale
        attn = attn.softmax(dim=-1)
        attn = self.attn_drop(attn)

        x = (attn @ v).transpose(1, 2).reshape(B, N, C)
        x = self.proj(x)
        x = self.proj_drop(x)
        return x

In [ ]:
class TileVit(nn.Module):
    """
    'efficientnet_b0' -> https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-weights/efficientnet_b0_ra-3dd342df.pth
    """

    def __init__(self) -> None:
        super().__init__()

        self.num_patches = 10
        self.model = timm.create_model("efficientnet_b0", pretrained=True, num_classes=0)
        for i, _children in enumerate(self.model.children()):
            print(i, type(_children))
            if i <= 1000:
                print("Freezing Layer....")
                for param in _children.parameters():
                    param.requires_grad = False
                
        self.embed = 128

        self.linear_mapper = nn.Linear(1280, self.embed)

        self.attention = AttentionCust(self.embed, 2)
        self.class_token = nn.Parameter(torch.rand(1, self.embed))

        self.last_layer = nn.Linear(self.embed, 1)


        # [RANDOM INIT]
        initialize_weights(self.linear_mapper)
        initialize_weights(self.last_layer)
        # initialize_weights(self.class_token)


    def forward(self, x):

        n, pa, c, w, h = x.shape

        # (1) - Featues extraction
        x = x.reshape(-1, 3, 224, 224)
        x = self.model(x)

        # (2) projection -> 64
        tokens = self.linear_mapper(x)

        tokens = tokens.reshape(-1, pa, self.embed)

        # (3) adding class token
        tokens = torch.stack([torch.vstack([self.class_token, tokens[i]]) for i in range(len(tokens))])

        # (4) adding positional embedding
        # tokens += get_positional_embedding(11, self.embed).repeat(n, 1, 1).to('cuda')

        tokens = self.attention(tokens)

        tokens = tokens[:, 0]

        tokens = self.last_layer(tokens)
        
        return tokens

In [ ]:
class TileVitModule(pl.LightningModule):

    def __init__(self):
        super().__init__()
        self.model = TileVit()

    def forward(self, x):
        return self.model(x)

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self.model(x)
        loss = F.binary_cross_entropy_with_logits(y_hat, y)
        self.log("train_loss", loss)
        return loss

    def on_train_epoch_start(self):
        print("Epoch:", self.current_epoch)

    def training_epoch_end(self, training_step_outputs):
        loss = torch.stack([o['loss'] for o in training_step_outputs]).cpu().mean()
        print("\n Training Loss:", loss)

    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self.model(x)
        loss = F.binary_cross_entropy_with_logits(y_hat, y)
        self.log("val_loss", loss)
        return loss


    def validation_epoch_end(self, validation_step_outputs):
        loss = torch.stack(validation_step_outputs).cpu().mean()
        print("\n Validation Loss:", loss)


    def configure_optimizers(self):
        optimizer = optim.AdamW(self.parameters(), lr=0.001)
        # scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=10, factor=0.1)
        # return {"optimizer": optimizer, "lr_scheduler": scheduler, "monitor": "val_loss"}
        return optimizer


In [ ]:
from pytorch_lightning.callbacks import TQDMProgressBar, ModelCheckpoint
from tqdm import tqdm
import sys

In [ ]:
# FIX : https://stackoverflow.com/questions/59455268/how-to-disable-progress-bar-in-pytorch-lightning
class LitProgressBar(TQDMProgressBar):

    def init_validation_tqdm(self):
        bar = tqdm(            
            disable=True,            
        )
        return bar

In [ ]:
# FIX SEED
random.seed(0)
torch.manual_seed(0)
np.random.seed(0)
pl.seed_everything(0)

model = TileVitModule()

train_split_loader = DataLoader(train_split_ds,batch_size=8, shuffle=True)

val_split_ds = TilesDataset(val_split_df, val_transform)
val_split_loader = DataLoader(val_split_ds,batch_size=8)

test_num += 1

dirpath = f'/content/train_{test_num}'
print("Saved AT:", dirpath)


checkpoint_callback = ModelCheckpoint(
    monitor='val_loss',
    dirpath=dirpath,
    filename='sample-mnist-{epoch:02d}-{val_loss:.4f}',
    save_top_k=2
)

trainer = pl.Trainer(
    accelerator="auto", 
    max_epochs=50,
    callbacks=[LitProgressBar(refresh_rate=2), checkpoint_callback]
)

trainer.fit(model, train_dataloaders=train_split_loader, val_dataloaders=val_split_loader)

In [ ]:
from pathlib import Path
import torch

In [ ]:
checkpoint_path = Path("/content/train_101/sample-mnist-epoch=48-val_loss=0.3604.ckpt")
print("Load from:", checkpoint_path)

saved_pt = str(checkpoint_path.parent / checkpoint_path.stem) + ".pt"
print("Saved At:", saved_pt)

load_for_save = TileVitModule.load_from_checkpoint(checkpoint_path)
script = load_for_save.to_torchscript()
torch.jit.save(script, saved_pt)

In [ ]:
loaded = torch.jit.load(saved_pt)
loaded.to('cuda')
loaded.eval()
pass

In [ ]:
val_split_ds = TilesDataset(val_split_df, val_transform)
print(len(val_split_ds))
val_split_loader = DataLoader(val_split_ds,batch_size=8)

true_ls = []
pred_ls = []

for i, data in enumerate(val_split_loader, 0):
    inputs, labels = data
    inputs = inputs.to('cuda')
    
    with torch.no_grad():
        pred = loaded(inputs).cpu()
        
    pred = torch.sigmoid(pred).numpy().ravel().tolist()
    labels = labels.numpy().ravel().tolist()
    
    true_ls.extend(labels)
    pred_ls.extend(pred)

In [ ]:
true_np = np.array(true_ls)
pred_np = np.array(pred_ls)
true_np.shape, pred_np.shape

threash = 0.6

pred_np[pred_np > threash] = 1
pred_np[pred_np < threash] = 0

In [ ]:
confusion_matrix(true_np, pred_np)

In [ ]:
print(classification_report(true_np, pred_np))

In [ ]:
1# %load_ext tensorboard

In [ ]:
# %tensorboard --logdir /content/lightning_logs